# Modelling the Tufa layer in the Ammer Valley Quaternary floodplain stratigraphy

This notebook explain step-by-step how to create a object based sedimentary structure model using HyVR and the python environment. The model will represent the tufa layer in the Quaternary floodplain sediments of the Ammer Valler, Tuebingen, Germany.

## Packages

Besides HyVR, we use numpy for numerical computing. Since we are interested in using this model for posterior flow simulations, we will use the capabilities in flopy, specially for handling the grid and exporting VTKs (3D render models).

In [2]:
from hyvr.tools import ferguson_curve  # used for the channel curve creation
from hyvr import channel  # channel object creation
from hyvr import trough  # trough creation
import scipy  # general scientific calculations
import flopy  # our modelling interface
import numpy as np  # general numerical functions and array manipulation

## Grid/Model creation

HyVR should work on any structured grid. One example would be creating a grid with `np.meshgrid`, the numpy function for grids. However, we are interested in flow simulations, and MODFLOW is the standard. The python interface, flopy has grid creation capabilities that can be easily translated to MODFLOW grids, thus we use that for our grid creation

In [3]:
# Model creation:
name = "ammer_V0602"
ws = "."
sim = flopy.mf6.MFSimulation(
    sim_name=name,
    exe_name="mf6",
    version="mf6",
    sim_ws=ws,
)
# Nam file
model_nam_file = "{}.nam".format(name)
# Groundwater flow object:
gwf = flopy.mf6.ModflowGwf(
    sim,
    modelname=name,
    model_nam_file=model_nam_file,
    save_flows=True,
)
# Grid properties:
Lx = 2000  # problem lenght [m]
Ly = 600  # problem width [m]
H = 7  # aquifer height [m]
delx = 1.5  # block size x direction
dely = 1.5  # block size y direction
delz = 0.2  # block size z direction
nlay = int(H / delz)
ncol = int(Lx / delx)  # number of columns
nrow = int(Ly / dely)  # number of layers

# Flopy Discretizetion Objects (DIS)
dis = flopy.mf6.ModflowGwfdis(
    gwf,
    xorigin=0.0,
    yorigin=0.0,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=dely,
    delc=delx,
    top=7.0,
    botm=np.arange(H - delz, 0 - delz, -delz),
)

# Node property flow
k = 1e-5  # Model conductivity in m/s
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    icelltype=0,  # This we define the model as confined
    k=k,
)

# Acessing the grid
grid = gwf.modelgrid

# cell centers
centers = grid.xyzcellcenters

X = centers[0]
Y = centers[1]
Z = centers[2]

# broadcasting the X, Y to the same shape as Z (full grid shape)
X = np.broadcast_to(X, Z.shape)
Y = np.broadcast_to(Y, Z.shape)

## Modelling Sedimentary Structures

We base the framework on the original work from Bennett et al (2018), which has written the first version of HyVR. The framework is hierarchical, meaning that structures can be organized in different scales depending on the context, but that they may represent complex sedimentary archictectures.

From the analysis of the facies present in the tufa we came up with an achitecture formed by the following elements:

The tufa is likely transported from upstream. Seepage from supersaturated groundwater from the carbonate formation intersects the ammer river upstream and probably caused the precipitation of tufa particles in association with adequate flora in a wetland-like environment. Then river water would carry this sediment downstream along with organic matter (often preserved as plant remains), and deposit this material in the Ammer floodplain. We see different facies with varying amounts of tufa and organic matter, and varying composition of organic matter. Likely the system stabilized for periods of time, giving structure to different local wetland environments. Depending on the stability and preservation characteristics peat lenses would form, and the river channel shape would be preserved as gravel deposits. During sediment inflow periods, continuous input of sedimentation would make the river shape unstable, meaning it would not preserve its shape. The wetland environments would then receive tufa sedimentation and deposit mixes of tufa clasts and phytoclasts and organic matter. We think of this period as a succession of discontinuous lenses which are elongated in the direction of flow of different facies associated with different local wetland environments reworked by transport and deposition of external tufa particles.

Therefore, we can think of the system as an aggradational sequence of lenses of different mix composition between tufas and phytoclasts, organic particles, which comprises different facies recorded in the sedimentary analysis. At the end of the sequence we would have the reduction of sedimentary the sedimentary load, leading to a stable configuration where peat lenses would be preserved and the preservation of channel features. Upon the next increase of sedimentary load, the channel features would be preferentially filled with gravel particles, while the peat lenses then would be buried, and the sequence then repeats itself.

The algorithm is organized as such:

1. Define the sequence thicknesses (sampled from a distribution)
2. Over the thickness t:
    2.1. Iterate over each facies f associated with the aggradation period:
        2.1.1. generate lens of thickness t at a randomly sampled location and with reasonably chosen dimensions. Assign it to facies f
        2.1.2. repeat 2.1 until the proportion of facies f is slightly above the calculated proportion (since one object can erode the previous we use slightly bigger proportion in the algorithm).
    2.3. Generate lens of thickness t or max_peat_thickness at a randomly sampled location and with reasonably chosen dimension of the facies peat
    2.4. repeat 2.3 until proportion of peat is the same as the calculated proportion.
    2.5. Generate a channel starting on the left of the grid and on a randomly sampled high y value with thickness t or max_channel_thickness and width $\approx$ 4 m.
    2.6. Generate a channel startubg on the left of the grid (x=0) and on a randomly sampled low y values with thickness t or max_channel_thickness and width $\approx$ 4 m.
3. Add the base level by thickness t and repeat 2. unil the end of the sequence (7m high).


### Defining the sequence of thicknesses

We will assume an average thickness of 0.7 m. The first layer in the system is modelled deterministically. It is $\approx$ 0.4 m thickness and composed with light color tufa with fossil and low organic matter content. The remaining layers are modelled probabilistically.
 which means that in the sequence of 7 m, we randomly sample 10 thicknesses:

Below we have calculated a distribution function that randomly sampled thicknesses with the characteristics above:

In [37]:
# according to answer in : https://math.stackexchange.com/questions/291174/probability-distribution-of-the-subinterval-lengths-from-a-random-interval-divis
# The cumulative distribution function for intervals randomly sampled from the interval [0,a] is:
simulated_thickness = 7 - 0.4
n = 9
print(f"With the number of layers: {n}")
F = lambda t: 1 - ((simulated_thickness - t) / simulated_thickness) ** (n - 1)
# The probability density function is then:
f = (
    lambda t: (n - 1)
    / simulated_thickness
    * ((simulated_thickness - t) / simulated_thickness) ** (n - 2)
)
mean_thickness = scipy.integrate.quad(lambda t: t * f(t), 0, simulated_thickness)[0]
median_thickness = scipy.optimize.fsolve(lambda t: F(t) - 0.5, mean_thickness)[0]
# The median thickness would be:
print(f"The median thickness is:{median_thickness}")
# The mean thickness would be:
print(f"The mean thickness is:{mean_thickness}")


With the number of layers: 9
The median thickness is:0.5477733148491694
The mean thickness is:0.7333333333333332


The code above has calculated that a 9 layer model would on average produce a thickness agreeable with measured average of 0.7 meters.
To generate the model thicknesses we can just generate random samples on the interval from 0 to the total modelled thickness. Unfortunately, due to cell size restrictions, we cannot model layers that are too small (< 0.3 m), therefore we iterate unil we have a thickness model with layers that are bigger than 0.3:

In [34]:
min_thick = 0
while min_thick < 0.3:
    zs = np.random.uniform(0, simulated_thickness, size=n - 1)
    ordered_zs = np.sort(zs)
    ordered_zs = np.append(ordered_zs, simulated_thickness)
    ordered_zs = np.append(0, ordered_zs)
    thicknesses = np.diff(ordered_zs)
    min_thick = np.min(thicknesses)


In [36]:
print(f"The thickness array is:{thicknesses})")
print(f"The minimum thickness is:{min_thick}")
print(f"The total thickness is:{np.sum(thicknesses)}")
print(f"The mean thickness is:{np.mean(thicknesses)}")
print(f"THe number of layers is:{len(thicknesses)}")


The thickness array is:[0.4732641  0.73957832 0.91983533 0.94173358 0.56025054 0.46504586
 0.33035965 1.12023074 1.04970188])
The minimum thickness is:0.33035965221052077
The total thickness is:6.6
The mean thickness is:0.7333333333333333
THe number of layers is:9


In [38]:
testf = np.empty((nrow, ncol), dtype=np.int32)
np.unique(testf)


array([0])

### Creating and running the sedimentary structure algorithm

In [ ]:
facies_tufa = np.array([2, 3, 4, 5], dtype=np.int32)
facies = np.empty_like(Z, dtype=np.int32)
z_0 = 0.0
for thick in thicknesses:
    ## initial tufa sheets: they are modelled as very elongated ellipsoids representing discontinuous layers
    p_t23 = 0
    while p_t23 < 0.3:
        x_c = np.random.uniform(0, 2000)
        y_c = np.random.uniform(0, 600)
        z_c = z_0 + thick + np.random.uniform(-0.2, 0)
        a = np.random.uniform(200, 400)
        b = np.random.uniform(100, 200)
        c = thick
        azim = np.random.uniform(70, 110)
        facies_trough, dip_dir_trough, dip_trough = trough(
            X,
            Y,
            Z,
            center_coords=np.array([x_c, y_c, z_c]),
            dims=np.array([a, b, c]),
            azim=azim,
            facies=np.array([2]),
        )
        facies[facies_trough != -1] = facies_trough[facies_trough != -1]
        # k[facies_trough != -1] = np.random.lognormal(mu_tufa, sigma=sigma_tufa)
        logic_tufa = (Z >= z_0) & (Z <= z_0 + thick)
        p_t23 = np.sum(facies[logic_tufa] == 2) / np.sum(logic_tufa)

    p_t5 = 0
    while p_t5 < 0.3:
        x_c = np.random.uniform(0, 2000)
        y_c = np.random.uniform(0, 600)
        z_c = z_0 + thick + np.random.uniform(-0.2, 0)
        a = np.random.uniform(200, 400)
        b = np.random.uniform(100, 200)
        c = thick  # thickness until the original base (more or less)
        azim = np.random.uniform(70, 110)
        facies_trough, dip_dir_trough, dip_trough = trough(
            X,
            Y,
            Z,
            center_coords=np.array([x_c, y_c, z_c]),
            dims=np.array([a, b, c]),
            azim=azim,
            facies=np.array([5]),
        )
        facies[facies_trough != -1] = facies_trough[facies_trough != -1]
        # k[facies_trough != -1] = np.random.lognormal(mu_moss, sigma=sigma_moss)
        logic_tufa = (Z >= z_0) & (Z <= z_0 + thick)
        p_t5 = np.sum(facies[logic_tufa] == 5) / np.sum(logic_tufa)
    p_t6 = 0
    while p_t6 < 0.2:
        x_c = np.random.uniform(0, 2000)
        y_c = np.random.uniform(0, 600)
        z_c = z_0 + thick + np.random.uniform(-0.2, 0)
        a = np.random.uniform(200, 400)
        b = np.random.uniform(100, 200)
        c = thick  # thickness until the original base (more or less)
        azim = np.random.uniform(70, 110)
        facies_trough, dip_dir_trough, dip_trough = trough(
            X,
            Y,
            Z,
            center_coords=np.array([x_c, y_c, z_c]),
            dims=np.array([a, b, c]),
            azim=azim,
            facies=np.array([6]),
        )
        facies[facies_trough != -1] = facies_trough[facies_trough != -1]
        # k[facies_trough != -1] = np.random.lognormal(mu_moss, sigma=sigma_moss)
        logic_tufa = (Z >= z_0) & (Z <= z_0 + thick)
        p_t6 = np.sum(facies[logic_tufa] == 6) / np.sum(logic_tufa)
    p_t7 = 0
    while p_t7 < 0.3:
        x_c = np.random.uniform(0, 2000)
        y_c = np.random.uniform(0, 600)
        z_c = z_0 + thick + np.random.uniform(-0.2, 0)
        a = np.random.uniform(200, 400)
        b = np.random.uniform(100, 200)
        c = thick  # thickness until the original base (more or less)
        azim = np.random.uniform(70, 110)
        facies_trough, dip_dir_trough, dip_trough = trough(
            X,
            Y,
            Z,
            center_coords=np.array([x_c, y_c, z_c]),
            dims=np.array([a, b, c]),
            azim=azim,
            facies=np.array([7]),
        )
        facies[facies_trough != -1] = facies_trough[facies_trough != -1]
        # k[facies_trough != -1] = np.random.lognormal(mu_moss, sigma=sigma_moss)
        logic_tufa = (Z >= z_0) & (Z <= z_0 + thick)
        p_t7 = np.sum(facies[logic_tufa] == 7) / np.sum(logic_tufa)

    # facies 4 is the background facies, we make sure it has a minimum presence with more sheets if necessary:
    p_t4 = np.sum(facies[logic_tufa] == 4) / np.sum(logic_tufa) + np.sum(
        facies[logic_tufa] == 0
    ) / np.sum(logic_tufa)
    while p_t4 < 0.4:
        x_c = np.random.uniform(0, 2000)
        y_c = np.random.uniform(0, 600)
        z_c = z_0 + thick + np.random.uniform(-0.2, 0)
        a = np.random.uniform(200, 400)
        b = np.random.uniform(100, 200)
        c = thick
        azim = np.random.uniform(70, 110)
        facies_trough, dip_dir_trough, dip_trough = trough(
            X,
            Y,
            Z,
            center_coords=np.array([x_c, y_c, z_c]),
            dims=np.array([a, b, c]),
            azim=azim,
            facies=np.array([4]),
        )
        facies[facies_trough != -1] = facies_trough[facies_trough != -1]

        logic_tufa = (Z >= z_0) & (Z <= z_0 + thick)
        p_t4 = np.sum(facies[logic_tufa] == 4) / np.sum(logic_tufa) + np.sum(
            facies[logic_tufa] == 0
        ) / np.sum(logic_tufa)

    # peat lenses:
    peat = 0.0
    # peat lenses
    while peat < 0.15:
        x_c = np.random.uniform(0, 2000)
        y_c = np.random.uniform(0, 600)
        z_c = z_0 + thick + np.random.uniform(-0.2, 0)
        a = np.random.uniform(100, 200)
        b = np.random.uniform(70, 100)
        azim = np.random.uniform(60, 120)
        if thick > 0.7:
            peat_depth = 0.7
        else:
            peat_depth = thick
        c = peat_depth

        facies_trough, dip_dir_trough, dip_trough = trough(
            X,
            Y,
            Z,
            center_coords=np.array([x_c, y_c, z_c]),
            dims=np.array([a, b, c]),
            azim=azim,
            facies=np.array([8]),
        )
        facies[facies_trough != -1] = facies_trough[facies_trough != -1]

        logic_peat = (Z >= z_0) & (Z <= z_0 + thick)
        peat = np.sum(facies[logic_peat] == 8) / np.sum(logic_peat)
        print(peat)
    # channels
    channel_curve_1 = ferguson_curve(
        h=0.1,
        k=np.pi / 200,
        eps_factor=(np.pi / 1.5) ** 2,
        flow_angle=0.0,
        s_max=4000,
        xstart=0,
        ystart=25,
    )
    y_shift_1 = np.random.uniform(400, 500)
    channel_1 = np.c_[channel_curve_1[0], channel_curve_1[1] + y_shift_1]
    if thick > 0.6:
        depth = 0.6
    else:
        depth = thick
    channel_f, channel_dip_dir, channel_dip = channel(
        X,
        Y,
        Z,
        z_top=z_0 + thick,
        curve=channel_1,
        parabola_pars=np.array([4, depth]),
        facies=np.array([11]),
    )
    facies[channel_f != -1] = channel_f[channel_f != -1]

    channel_curve_2 = ferguson_curve(
        h=0.1,
        k=np.pi / 200,
        eps_factor=(np.pi / 1.5) ** 2,
        flow_angle=0.0,
        s_max=4000,
        xstart=0,
        ystart=25,
    )
    y_shift_2 = np.random.uniform(40, 150)
    channel_2 = np.c_[channel_curve_2[0], channel_curve_2[1] + y_shift_2]
    channel_f, channel_dip_dir, channel_dip = channel(
        X,
        Y,
        Z,
        z_top=z_0 + thick,
        curve=channel_2,
        parabola_pars=np.array([4, depth]),
        facies=np.array([11]),
    )
    facies[channel_f != -1] = channel_f[channel_f != -1]

    # resetting z_0:
    z_0 += thick
